<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [7]:
!"C:\Program Files (x86)\GnuWin32\bin\head.exe" -1 stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [9]:
%%time
!"C:\Program Files (x86)\GnuWin32\bin\wc.exe" -l stackoverflow.10kk.tsv

10000000 stackoverflow.10kk.tsv
Wall time: 3min 18s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [1]:
import os
from tqdm import tqdm,tqdm_notebook
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. 10 Гб у меня обработались примерно за 2 минуты.

In [2]:
#!python preprocess.py stackoverflow.10kk.tsv stackoverflow.vw
%run  preprocess.py stackoverflow.10kk.tsv stackoverflow.vw

in: stackoverflow.10kk.tsv out: stackoverflow.vw


100%|████████████████████████████████████████████████████████████████████| 10000000/10000000 [21:53<00:00, 7611.84it/s]


5610942 lines was corrupted
4389058 lines was fine


In [11]:
%%time
!"C:\Program Files (x86)\GnuWin32\bin\wc.exe" -l stackoverflow.vw

4389058 stackoverflow.vw
Wall time: 4min 20s


In [2]:
!"C:\Program Files (x86)\GnuWin32\bin\gzip.exe" stackoverflow.10kk.tsv

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [31]:
%%time
!"C:\Program Files (x86)\GnuWin32\bin\head.exe" -n 1463018 stackoverflow.vw > stackoverflow_train.vw
!"C:\Program Files (x86)\GnuWin32\bin\head.exe" -n 2926036 stackoverflow.vw | "C:\Program Files (x86)\GnuWin32\bin\tail.exe"  -n 1463018 > stackoverflow_valid.vw
!"C:\Program Files (x86)\GnuWin32\bin\tail.exe" -n 1463018 stackoverflow.vw > stackoverflow_test.vw

!"C:\Program Files (x86)\GnuWin32\bin\cat.exe" stackoverflow_test.vw | "C:\Program Files (x86)\GnuWin32\bin\cut.exe" -f1 -d "|" > stackoverflow_test_labels.txt
!"C:\Program Files (x86)\GnuWin32\bin\cat.exe" stackoverflow_valid.vw | "C:\Program Files (x86)\GnuWin32\bin\cut.exe" -f1 -d "|" > stackoverflow_valid_labels.txt


Wall time: 33.2 s


In [33]:
%%time
!"C:\Program Files (x86)\GnuWin32\bin\wc.exe" -l stackoverflow_train.vw
!"C:\Program Files (x86)\GnuWin32\bin\wc.exe" -l stackoverflow_valid.vw
!"C:\Program Files (x86)\GnuWin32\bin\wc.exe" -l stackoverflow_test.vw
!"C:\Program Files (x86)\GnuWin32\bin\wc.exe" -l stackoverflow_valid_labels.txt
!"C:\Program Files (x86)\GnuWin32\bin\wc.exe" -l stackoverflow_test_labels.txt

1463018 stackoverflow_train.vw
1463018 stackoverflow_valid.vw
1463018 stackoverflow_test.vw
1463018 stackoverflow_valid_labels.txt
1463018 stackoverflow_test_labels.txt
Wall time: 1min 21s


### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: `loss_function=hinge`, `bit_precision`=28 и `seed`=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [ ]:
passes = [1, 3, 5]
ngram = [1, 2, 3]

params = [(p, n) for p in passes for n in ngram]

with open('stackoverflow_valid_labels.txt', 'r') as file:
    valid_labels = [int(label.strip()) for label in file]

best_score = 0
best_params = None

for (p, n) in tqdm_notebook(params):
    print('learn: passes={}; ngram={}'.format(p, n))
    !"C:\Program Files\VowpalWabbit\vw.exe" --oaa 10 -d stackoverflow_train.vw --loss_function=hinge --bit_precision 28 --random_seed 17 \
        -f valid_model.vw --cache --passes $p --ngram $n 
    print('test on valid...')
    !"C:\Program Files\VowpalWabbit\vw.exe" -i valid_model.vw -t -d stackoverflow_valid.vw \
        -p valid_predictions.txt
    with open('valid_predictions.txt', 'r') as file:
        test_labels = [int(label.strip()) for label in file]
    acc = accuracy_score(valid_labels, test_labels)
    print('accuracy={}'.format(acc))
    if acc > best_score:
        best_params = (p, n)
        best_score = acc

A Jupyter Widget

learn: passes=1; ngram=1


Generating 1-grams for all namespaces.
final_regressor = valid_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = stackoverflow_train.vw.cache
Reading datafile = stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      129
0.500000 1.000000            2            2.0        4        1       53
0.750000 1.000000            4            4.0        7        1       55
0.750000 0.750000            8            8.0        7        1       76
0.750000 0.750000           16           16.0        7        7      153
0.812500 0.875000           32           32.0        7        2      150
0.765625 0.718750           64           64.0        3        3      188
0.664063 0.562500          128          128.0        1        5       21
0.578125 0.492188   

test on valid...


Generating 1-grams for all namespaces.
only testing
predictions = valid_predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        2        2      138
0.000000 0.000000            2            2.0        7        7       61
0.000000 0.000000            4            4.0        5        5      202
0.000000 0.000000            8            8.0        7        7      106
0.000000 0.000000           16           16.0        6        6      229
0.000000 0.000000           32           32.0        2        2      327
0.062500 0.125000           64           64.0        5        5      667
0.054688 0.046875          128          128.0        2        2      103
0.097656 0.140625          256          

accuracy=0.9100127271161393
learn: passes=1; ngram=2


In [2]:
def train_vw_model(train_vw_file, model_filename, num_classes=10,
                   ngram=1, loss_function='hinge',
                   bit_precision=28, passes=1,
                   seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('"C:/Program Files/VowpalWabbit/vw.exe" --oaa {num_classes} {train_vw_file} ' + 
                      '-f {model_filename} -b {bit_precision} ' +
                      '--loss_function {loss_function} --random_seed {seed}').format(
                       num_classes=num_classes, train_vw_file=train_vw_file, 
                        loss_function=loss_function,
                       model_filename=model_filename, 
        bit_precision=bit_precision, seed=seed)
    if ngram > 1:
         vw_call_string += ' --ngram={}'.format(ngram)
            
    if passes > 1:
         vw_call_string += ' -k --passes={} --cache_file {}'.format(passes, 
                            model_filename.replace('.vw', '.cache'))
    if quiet:
        vw_call_string += ' --quiet'
    
    
    print(vw_call_string) 
    res = os.system(vw_call_string)
    print('Success. Elapsed: {} sec.'.format(round(time() - init_time, 2))
          if not res else 'Failed.')

In [3]:
def test_vw_model(model_filename, test_vw_file, prediction_filename,
                  true_labels, seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('"C:/Program Files/VowpalWabbit/vw.exe" -t -i {model_filename} {test_vw_file} ' + 
                       '-p {prediction_filename} --random_seed {seed}').format(
                       model_filename=model_filename, test_vw_file=test_vw_file, 
                       prediction_filename=prediction_filename, seed=seed)
    if quiet:
        vw_call_string += ' --quiet'
        
    print(vw_call_string) 
    res = os.system(vw_call_string)
    
    if not res: # the call resulted OK
        vw_pred = np.loadtxt(prediction_filename)
        print("Accuracy: {}%. Elapsed: {} sec.".format(
            round(100 * accuracy_score(true_labels, vw_pred), 2), 
            round(time() - init_time, 2)))
    else:
        print('Failed.')

In [4]:
y_valid = np.loadtxt('stackoverflow_valid_labels.txt')
y_test = np.loadtxt('stackoverflow_test_labels.txt')

In [5]:
import itertools

In [ ]:
for i, (ngram, passes) in tqdm(enumerate(itertools.product([1,2,3], 
                                                      [1,3,5]))):
    train_vw_model('stackoverflow_train.vw',
                   'vw_model{}_part.vw'.format(i), 
                   ngram=ngram, passes=passes,
                   loss_function='hinge',
                   num_classes=10, bit_precision=28, 
                   seed=17, quiet=True)
    test_vw_model(model_filename='vw_model{}_part.vw'.format(i), 
              test_vw_file='stackoverflow_valid.vw', 
              prediction_filename='vw_valid_pred{}.csv'.format(i),
              true_labels=y_valid, seed=17)

0it [00:00, ?it/s]

"C:/Program Files/VowpalWabbit/vw.exe" --oaa 10 stackoverflow_train.vw -f vw_model0_part.vw -b 28 --loss_function hinge --random_seed 17 --quiet
Success. Elapsed: 266.07 sec.
"C:/Program Files/VowpalWabbit/vw.exe" -t -i vw_model0_part.vw stackoverflow_valid.vw -p vw_valid_pred0.csv --random_seed 17 --quiet
Accuracy: 91.0%. Elapsed: 89.51 sec.


1it [05:58, 358.90s/it]

"C:/Program Files/VowpalWabbit/vw.exe" --oaa 10 stackoverflow_train.vw -f vw_model1_part.vw -b 28 --loss_function hinge --random_seed 17 -k --passes=3 --cache_file vw_model1_part.cache --quiet


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)